In [9]:
import numpy as np
import xarray as xr
from scipy.interpolate import RegularGridInterpolator

def resize_nc_variables(nc_file, new_shape=(60, 80)):
    dataset = xr.open_dataset(nc_file)
    resized_data = {}

    for var_name in dataset.variables:
        data = dataset[var_name].values
        data[data == -32768.0] = np.nan  # 替换标记值为 NaN

        if data.ndim == 3:  # 三维数据
            depth, lat, lon = dataset[var_name].dims
            if var_name in ['u', 'v']:
                if var_name == 'u':
                    lat, lon = 'latu', 'lonu'
                elif var_name == 'v':
                    lat, lon = 'latv', 'lonv'
            old_coords = (dataset[depth].values, dataset[lat].values, dataset[lon].values)
            new_coords = np.meshgrid(dataset[depth].values,
                                     np.linspace(dataset[lat].values.min(), dataset[lat].values.max(), new_shape[0]),
                                     np.linspace(dataset[lon].values.min(), dataset[lon].values.max(), new_shape[1]),
                                     indexing='ij')
            interpolator = RegularGridInterpolator(old_coords, data, bounds_error=False, fill_value=np.nan)
            resized_data[var_name] = interpolator(tuple(map(np.ravel, new_coords))).reshape(len(dataset[depth]), *new_shape)

        elif data.ndim == 2:  # 二维数据
            lat, lon = dataset[var_name].dims
            old_coords = (dataset[lat].values, dataset[lon].values)
            new_coords = np.meshgrid(np.linspace(dataset[lat].values.min(), dataset[lat].values.max(), new_shape[0]),
                                     np.linspace(dataset[lon].values.min(), dataset[lon].values.max(), new_shape[1]),
                                     indexing='ij')
            interpolator = RegularGridInterpolator(old_coords, data, bounds_error=False, fill_value=np.nan)
            resized_data[var_name] = interpolator(tuple(map(np.ravel, new_coords))).reshape(new_shape)
        
    return resized_data

# 示例调用
file_path="E:/Dataset/redos/Subset_1.0_1995/subset_19950214.nc"
resized_data = resize_nc_variables(file_path)

# 验证每个变量的形状
for var_name, data in resized_data.items():
    print(f"{var_name} 的插值后形状: {data.shape}")
    num_samples=5
    print(f"{var_name} 的前 {num_samples} 个数据:\n{resized_data[var_name].flat[:num_samples]}")

import numpy as np

def count_valid_and_nan(data):
    """
    统计数据中的有效值和NaN值的数量。
    
    参数:
        data (np.ndarray): 需要统计的数据。
        
    返回:
        valid_count (int): 有效值的数量。
        nan_count (int): NaN值的数量。
    """
    valid_count = np.sum(~np.isnan(data))
    nan_count = np.sum(np.isnan(data))
    return valid_count, nan_count

# 示例数据加载和统计
# 假设 data 是你的数据字典，包含 'u' 和 'v' 变量

# 统计 'u' 数据
u_valid, u_nan = count_valid_and_nan(resized_data['u'])
print(f"u 数据中的有效值数量: {u_valid}")
print(f"u 数据中的NaN值数量: {u_nan}")

# 统计 'v' 数据
v_valid, v_nan = count_valid_and_nan(resized_data['v'])
print(f"v 数据中的有效值数量: {v_valid}")
print(f"v 数据中的NaN值数量: {v_nan}")


t 的插值后形状: (24, 60, 80)
t 的前 5 个数据:
[4.94823551 4.92691479 4.92760798 4.95510073 4.95374738]
s 的插值后形状: (24, 60, 80)
s 的前 5 个数据:
[34.58226013 34.57994278 34.57947458 34.58125754 34.58079041]
zeta 的插值后形状: (60, 80)
zeta 的前 5 个数据:
[0.75479156 0.74220871 0.72920666 0.71569428 0.70158887]
u 的插值后形状: (24, 60, 80)
u 的前 5 个数据:
[nan nan nan nan nan]
v 的插值后形状: (24, 60, 80)
v 的前 5 个数据:
[nan nan nan nan nan]
u 数据中的有效值数量: 48975
u 数据中的NaN值数量: 66225
v 数据中的有效值数量: 49628
v 数据中的NaN值数量: 65572


In [ ]:
file_path="E:/Dataset/redos/Subset_1.0_1995/subset_19950214.nc"
from datetime import datetime, timedelta
from scipy.ndimage import zoom
import numpy as np
import xarray as xr
from scipy.interpolate import RegularGridInterpolator

def interpolate_3d_data(nc_file, var_name, target_shape=(60, 80), anomaly_value=-32768.0):
    """
    插值三维数据并忽略异常值。

    参数:
    - nc_file (str): NetCDF 文件路径。
    - var_name (str): 需要插值的变量名称。
    - target_shape (tuple): 目标纬度和经度的形状 (height, width)。
    - anomaly_value (float): 异常值的标记，默认值为 -32768.0。

    返回:
    - new_data (numpy.ndarray): 插值后的数据。
    """
    # 打开NetCDF文件并读取变量数据
    dataset = xr.open_dataset(nc_file)
    data = dataset[var_name].values

    # 将标记值替换为 NaN
    data[data == anomaly_value] = np.nan

    # 获取原始数据的深度、纬度和经度
    depth = dataset['depth'].values
    lat = dataset['lat'].values
    lon = dataset['lon'].values

    # 构建新的网格
    new_lat = np.linspace(lat.min(), lat.max(), target_shape[0])
    new_lon = np.linspace(lon.min(), lon.max(), target_shape[1])

    # 进行插值，忽略 NaN 值
    interpolator = RegularGridInterpolator((depth, lat, lon), data, bounds_error=False, fill_value=np.nan)
    new_data = interpolator(np.meshgrid(depth, new_lat, new_lon, indexing='ij'))

    # 插值完成后，进一步处理 NaN 值，例如填充或删除
    # new_data = np.nan_to_num(new_data)  # 将所有 NaN 填充为 0

    return new_data

# 示例调用
nc_file = 'your_nc_file.nc'
var_name = 's'
interpolated_data = interpolate_3d_data(nc_file, var_name)
print(interpolated_data.shape)
# 示例调用
nc_file = "/path/to/your/ncfile.nc"
# new_data_dict = interpolate_nc_data(nc_file)

def load_all_nc_data(path, start_year, end_year):
    data_dict = {}
    data_dict_Jan={}
    current_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year + 1, 1, 1)
    while current_date < end_date:
        date_str = current_date.strftime('%Y%m%d')
        nc_file = path + '/subset_' + date_str + '.nc'
        # 当月份大于2时停止循环
        if current_date.month > 3:
            print("月份大于3，停止循环。")
            break
        # 读取 NetCDF 文件
        dataset = xr.open_dataset(nc_file)
        target_shape = (60, 80)

        # 计算缩放因子
        zoom_factors = (target_shape[0] / img.shape[0], target_shape[1] / img.shape[1])
        
        # 使用双线性插值进行调整
        resized_img = zoom(img, zoom_factors, order=1)  # order=1表示双线性插值

        print(resized_img.shape)  # 输出应该是(60, 80)
        for var_name in dataset.variables:
            data = dataset[var_name].values
            data[data == -32768.0] = np.nan  # 替换标记值
            if current_date.month == 1:
                if var_name not in data_dict_Jan:
                    data_dict_Jan[var_name] = []  # 初始化为列表
                # 处理1月份的数据
                data_dict_Jan[var_name].append(data)
            else:
                if var_name not in data_dict:
                    data_dict[var_name] = []  # 初始化为列表
                # 处理其他月份的数据
                data_dict[var_name].append(data)

        current_date += timedelta(days=1)

        # 将列表转换为数组
    for var_name in data_dict:
        data_dict[var_name] = np.array(data_dict[var_name])
        data_dict_Jan[var_name] = np.array(data_dict_Jan[var_name])
    return data_dict, data_dict_Jan
file_path="E:/Dataset/redos/Subset_1.0_1995"
# 假设原始数据为img，形状为(28, 52)
img = np.random.rand(28, 52)  # 示例数据

# 目标尺寸

